<a href="https://colab.research.google.com/github/rayaneghilene/Project-Runner/blob/main/Copy_of_TP3_P1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 3, Part 1 : N_Gram on WikiText

## Download WikiText-2 dataset

In [ ]:
! wget https://raw.githubusercontent.com/pytorch/examples/master/word_language_model/data/wikitext-2/train.txt
! wget https://raw.githubusercontent.com/pytorch/examples/master/word_language_model/data/wikitext-2/valid.txt
! wget https://raw.githubusercontent.com/pytorch/examples/master/word_language_model/data/wikitext-2/test.txt

--2023-04-02 11:37:16--  https://raw.githubusercontent.com/pytorch/examples/master/word_language_model/data/wikitext-2/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10797148 (10M) [text/plain]
Saving to: ‘train.txt’

train.txt           100%[===================>]  10.30M  --.-KB/s    in 0.06s   

2023-04-02 11:37:17 (172 MB/s) - ‘train.txt’ saved [10797148/10797148]

--2023-04-02 11:37:17--  https://raw.githubusercontent.com/pytorch/examples/master/word_language_model/data/wikitext-2/valid.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting resp

## Helper function

In [ ]:
from collections import Counter, defaultdict
import math
import copy
import random
import operator

flatten = lambda l: [item for sublist in l for item in sublist]

# some helper functions
def prepare_data(filename):
    data = [l.strip().lower().split() + ['</s>'] for l in open(filename) if l.strip()]
    corpus = flatten(data)
    vocab = set(corpus)
    return vocab, data

# About this lab

In this notebook, you will work with the N-Gram Language Model on the Wiki Text Dataset. This first lab should take you approximatively 1h.

<div align ="center"><img src="https://www.w3.org/TR/ngram-spec/tree2.gif"></div>

# A - The N-Gram Language Model

A language model assigns a probability to each possible next word given a history of previous words (context). 

<div align='center'> $P(w_t|w_{t-1}, w_{t-2}, ... , w_1)$ </div>



## Markov Assumption

Since calculating the probability of the whole sentence is not feasible, the Markov Assumption is introduced.

It assumes that each next word only depends on the previous K words (in an N-Gram language model, K = N-1).
- Unigram: $P(w_t|w_{t-1}, w_{t-2}, ... , w_1) = P(w_t)$
- Bigram: $P(w_t|w_{t-1}, w_{t-2}, ... , w_1) = P(w_t|w_{t-1}) $
- Trigram: $P(w_t|w_{t-1}, w_{t-2}, ... , w_1) = P(w_t|w_{t-1}, w_{t-2})$

For an N-Gram language model, the probability is calculated by counting the frequency:

$P(w_t|w_{t-1}, w_{t-2}, ... ,w_{t-N+1}) = \frac{C(w_t, w_{t-1}, w_{t-2}, ... ,w_{t-N+1} )}{C(w_{t-1}, w_{t-2}, ... ,w_{t-N+1})}$


We provide you the code of the NGram Language Model.

* Understand the code

In [ ]:

class NGramLM():
    def __init__(self, N):
        self.N = N
        self.vocab = set()
        self.data = []
        self.prob = {}
        self.counts = defaultdict(Counter)
    
    # For N = 1, the probability is stored in a dict   P = prob[next_word]
    # For N > 1, the probability is in a nested dict   P = prob[context][next_word]
    def train(self, vocab, data, smoothing_k=0):
        self.vocab = vocab
        self.data = data
        self.smoothing_k = smoothing_k

        if self.N == 1:
            self.counts = Counter(flatten(data))
            self.prob = self.get_prob(self.counts)
        else:
            self.vocab.add('<s>')
            counts = self.count_ngram()
            
            self.prob = {}
            for context, counter in counts.items():
                self.prob[context] = self.get_prob(counter)
    
    def count_ngram(self):
        counts = defaultdict(Counter)
        for sentence in self.data:
            sentence = (self.N - 1) * ['<s>'] + sentence 
            for i in range(len(sentence)-self.N+1):
                context = sentence[i:i+self.N-1]
                context = " ".join(context)
    
    # For N = 1, the probability is stored in a dict   P = prob[next_word
                word = sentence[i+self.N-1]
                counts[context][word] += 1

        self.counts = counts
        return counts
        
    # normalize counts into probability(considering smoothing)
    def get_prob(self, counter):
        total = float(sum(counter.values()))
        k = self.smoothing_k
    
    # For N = 1, the probability is stored in a dict   P = prob[next_word
        
        prob = {}
        for word, count in counter.items():
            prob[word] = (count + k) / (total + len(self.vocab) * k)
        return prob
        
    def get_ngram_logprob(self, word, seq_len, context=""):
        if self.N == 1 and word in self.prob.keys():
            return math.log(self.prob[word]) / seq_len
        elif self.N > 1 and not self._is_unseen_ngram(context, word):
            return math.log(self.prob[context][word]) / seq_len
        else:
            # assign a small probability to the unseen ngram
            # to avoid log of zero and to penalise unseen word or context
            return math.log(1/len(self.vocab)) / seq_len
        
    def get_ngram_prob(self, word, context=""):
        if self.N == 1 and word in self.prob.keys():
            return self.prob[word]
        elif self.N > 1 and not self._is_unseen_ngram(context, word):
            return self.prob[context][word]
        elif word in self.vocab and self.smoothing_k > 0:
            # probability assigned by smoothing
            return self.smoothing_k / (sum(self.counts[context].values()) + self.smoothing_k*len(self.vocab))
        else:
            # unseen word or context
            return 0
            
    # In this method, the perplexity is measured at the sentence-level, averaging over all sentences.
    # Actually, it is also possible to calculate perplexity by merging all sentences into a long one.
    def perplexity(self, test_data):
        log_ppl = 0
        if self.N == 1:
            for sentence in test_data:
                for word in sentence:
                    log_ppl += self.get_ngram_logprob(word=word, seq_len=len(sentence))
        else:
            for sentence in test_data:
                for i in range(len(sentence)-self.N+1):
                    context = sentence[i:i+self.N-1]
                    context = " ".join(context)
                    word = sentence[i+self.N-1]
                    log_ppl += self.get_ngram_logprob(context=context, word=word, seq_len=len(sentence))
        log_ppl /= len(test_data)
        ppl = math.exp(-log_ppl)
        return ppl
    
    def _is_unseen_ngram(self, context, word):
        if context not in self.prob.keys() or word not in self.prob[context].keys():
            return True
        else:
            return False
    
    # generate the most probable k words
    def generate_next(self, context, k):
        context = (self.N-1) * '<s> ' + context
        context = context.split()
        ngram_context_list = context[-self.N+1:]
        ngram_context = " ".join(ngram_context_list)
        
        if ngram_context in self.prob.keys():
            candidates = self.prob[ngram_context]
            most_probable_words = sorted(candidates.items(), key=lambda kv: kv[1], reverse=True)
            for i in range(min(k, len(most_probable_words))):
                print(" ".join(context[self.N-1:])+" "+most_probable_words[i][0]+"\t P={}".format(most_probable_words[i][1]))
        else:
            print("Unseen context!")
            
    # generate the next n words with greedy search
    def generate_next_n(self, context, n):
        context = (self.N-1) * '<s> ' + context
        context = context.split()
        ngram_context_list = context[-self.N+1:]
        ngram_context = " ".join(ngram_context_list)
        
        for i in range(n):
            try:
                candidates = self.prob[ngram_context]
                most_likely_next = max(candidates.items(), key=operator.itemgetter(1))[0]
                context += [most_likely_next]
                ngram_context_list = ngram_context_list[1:] + [most_likely_next]
                ngram_context = " ".join(ngram_context_list)
            except:
                break
        print(" ".join(context[self.N-1:]))
    

# B -Train with toy dataset

As the model is created we can train a N-Gram LM. At this step, let's train a Bigram language model on the toy dataset.

* What preprocessing was done ?
* How many word do we have in our vocabulary ?

* The preprocessing here is to have our data as a form a 
matrix each word corresponds to an element of this matrix and each line is a sentence . 

* We have 7 words in our vocabulary 

In [ ]:
corpus = ["I like whipped cream",
         "I like candys",
         "I hate tomatoes"]
data = [l.strip().lower().split() + ['</s>'] for l in corpus if l.strip()]
vocab = set(flatten(data))

# TODO : Answer the questions
print(data)
print(vocab)

[['i', 'like', 'whipped', 'cream', '</s>'], ['i', 'like', 'candys', '</s>'], ['i', 'hate', 'tomatoes', '</s>']]
{'hate', 'tomatoes', 'candys', 'whipped', 'cream', '</s>', 'i', 'like'}


In [ ]:
def print_probability(lm):
    for context in lm.vocab:
        for word in lm.vocab:
            prob = lm.get_ngram_prob(word, context)
            print("P({}\t|{}) = {}".format(word, context, prob))
        print("--------------------------")

# C - Smoothing
Smoothing method is used to deal with the sparsity problem in N-Gram language modelling.
The probability mass is shifted towards the less frequent words.

For example, with an add-1 smoothing, the probability is calculated as:

$$P(w_t | context) = \frac{C(w_t, context)+1}{C(context)+|V|}$$



**Q1: What is the disadvantage of smoothing?**

A:Smoothing can lead to overfitting, where the language model becomes too specific to the training data and does not generalize well to new data. This can be particularly problematic if the training data is limited, as the language model may not capture the true distribution of N-grams in the language.

In [ ]:
lm = NGramLM(2)
lm.train(vocab, data, smoothing_k=0)

#print_probability(lm)
########################################################
# TODO: try with add-2 smoothing and see the probability
########################################################
lm.train(vocab, data, smoothing_k=2)

print_probability(lm)

P(hate	|hate) = 0.10526315789473684
P(tomatoes	|hate) = 0.15789473684210525
P(candys	|hate) = 0.10526315789473684
P(whipped	|hate) = 0.10526315789473684
P(cream	|hate) = 0.10526315789473684
P(</s>	|hate) = 0.10526315789473684
P(i	|hate) = 0.10526315789473684
P(<s>	|hate) = 0.10526315789473684
P(like	|hate) = 0.10526315789473684
--------------------------
P(hate	|tomatoes) = 0.10526315789473684
P(tomatoes	|tomatoes) = 0.10526315789473684
P(candys	|tomatoes) = 0.10526315789473684
P(whipped	|tomatoes) = 0.10526315789473684
P(cream	|tomatoes) = 0.10526315789473684
P(</s>	|tomatoes) = 0.15789473684210525
P(i	|tomatoes) = 0.10526315789473684
P(<s>	|tomatoes) = 0.10526315789473684
P(like	|tomatoes) = 0.10526315789473684
--------------------------
P(hate	|candys) = 0.10526315789473684
P(tomatoes	|candys) = 0.10526315789473684
P(candys	|candys) = 0.10526315789473684
P(whipped	|candys) = 0.10526315789473684
P(cream	|candys) = 0.10526315789473684
P(</s>	|candys) = 0.15789473684210525
P(i	|candys)

# D -Train on WikiText-2 dataset and Evaluate Perplexity


## Evaluating perplexity

We define the perplexity as 

<div align="center">
$ PPL(W) = P(w_1, w_2, ... , w_n)^{-\frac{1}{n}}$

$ log(PPL(W)) = -\frac{1}{n}\sum^n_{k=1}log(P(w_k|w_1, w_2, ... , w_{k-1}))$
</div>





**Q2: Why do we need to calculate log perplexity?**

A: The perplexity score is calculated by taking the exponential of the cross-entropy loss, which can lead to numerical instability when working with small probabilities. Taking the logarithm of the perplexity score converts the multiplication of probabilities into addition, which is more numerically stable.

In [ ]:
vocab, train_data = prepare_data('train.txt')
valid_vocab, valid_data = prepare_data('valid.txt')
test_vocab, test_data = prepare_data('test.txt')
print(len(vocab))

28912


In [ ]:
lm = NGramLM(3)

# Train the model using vocab and training data

lm.train(vocab,train_data)

In [ ]:
print(lm.perplexity(valid_data))
print(lm.perplexity(test_data))

387.1190374834364
310.6880368161233


# E - Generating sentences

With a pre-trained N-Gram language model, we can predict possible next words given context.

In [ ]:
# generate the most probable following words given the context
print(" ".join(valid_data[12]))

# actually the only useful context in the Trigram language model is ["where", "they"]
context = "the eggs hatch at night , and the larvae swim to the water surface where they"  
lm.generate_next(context, 3)


the eggs hatch at night , and the larvae swim to the water surface where they drift with the ocean currents , preying on <unk> . this stage involves three <unk> and lasts for 15 – 35 days . after the third moult , the juvenile takes on a form closer to the adult , and adopts a <unk> lifestyle . the juveniles are rarely seen in the wild , and are poorly known , although they are known to be capable of digging extensive burrows . it is estimated that only 1 larva in every 20 @,@ 000 survives to the <unk> phase . when they reach a carapace length of 15 mm ( 0 @.@ 59 in ) , the juveniles leave their burrows and start their adult lives . </s>
the eggs hatch at night , and the larvae swim to the water surface where they were	 P=0.12408759124087591
the eggs hatch at night , and the larvae swim to the water surface where they are	 P=0.08029197080291971
the eggs hatch at night , and the larvae swim to the water surface where they would	 P=0.051094890510948905


In [ ]:
# we can also generate with shorter contexts, even shorter than N-1

contexts = ["the eggs",
            "the",
            ""]
for context in contexts:
  lm.generate_next(context, 3)
  print("---")

the eggs hatch	 P=0.18181818181818182
the eggs of	 P=0.18181818181818182
the eggs are	 P=0.13636363636363635
---
the first	 P=0.03398926654740608
the <unk>	 P=0.020274299344066785
the episode	 P=0.015802027429934407
---
 =	 P=0.26132873311734756
 the	 P=0.14112004039214035
 in	 P=0.06353347077881095
---


In [ ]:
context = "the eggs hatch at night , and the larvae swim to the water surface where they"  

# generate the next n words with greedy search
lm.generate_next_n(context, 10)

# This is not a good method in practice,
# because wrong predictions in the early steps would introduce errors to the following predictions
lm.generate_next_n(context, 20)

the eggs hatch at night , and the larvae swim to the water surface where they were not able to get the part of the <unk>
the eggs hatch at night , and the larvae swim to the water surface where they were not able to get the part of the <unk> of the <unk> of the <unk> of the <unk> of


# F - Effect of N

**Q3: Why does the perplexity increase when N is large?**

A: because this results in sparsity(many of these N-grams are not present in the training data, resulting in sparse estimates of their probabilities) and overfitting issues that lead to higher perplexity.

In [ ]:
for n in range(1,6):
    lm = NGramLM(n)
    lm.train(vocab, train_data)
    print("************************")
    print("{}-gram LM perplexity on valid set: {}".format(n, lm.perplexity(valid_data)))
    print("{}-gram LM perplexity on test  set: {}".format(n, lm.perplexity(test_data)))

************************
1-gram LM perplexity on valid set: 599.6450225274759
1-gram LM perplexity on test  set: 553.0467265986567
************************
2-gram LM perplexity on valid set: 130.75949726991627
2-gram LM perplexity on test  set: 114.76592406312065
************************
3-gram LM perplexity on valid set: 387.1190374834364
3-gram LM perplexity on test  set: 310.6880368161233
************************
4-gram LM perplexity on valid set: 1091.9348578736633
4-gram LM perplexity on test  set: 846.7563102382584
************************
5-gram LM perplexity on valid set: 1558.823545905558
5-gram LM perplexity on test  set: 1201.746345143052


# G - Interpolation
In interpolation, we mix the probability estimates from all the n-gram estimators to alleviate the sparsity problem.

In [ ]:
class InterpolateNGramLM(NGramLM):
    
    def __init__(self, N):
        super(InterpolateNGramLM, self).__init__(N)
        self.ngram_lms = []
        self.lambdas = []
        
    def train(self, vocab, data, smoothing_k=0, lambdas=[]):
        assert len(lambdas) == self.N
        assert sum(lambdas) - 1 < 1e-9
        self.vocab = vocab
        self.lambdas = lambdas
        
        for i in range(self.N, 0, -1):
            # Instanciate the model
            lm = NGramLM(i)
            print(f"Training {i}-gram language model".format(i))
            lm.train(vocab, data, smoothing_k)
            self.ngram_lms.append(lm)
    
    def get_ngram_logprob(self, word, seq_len, context):
        prob = 0
        for i, (coef, lm) in enumerate(zip(self.lambdas, self.ngram_lms)):
            context_words = context.split()
            cutted_context = " ".join(context_words[-self.N + i + 1:])
            prob += coef * lm.get_ngram_prob(context=cutted_context, word=word)
        return math.log(prob) / seq_len
        

In [ ]:
###################################################
# Q5: tune your coefficients to decrease perplexity
###################################################
ilm = InterpolateNGramLM(3)
ilm.train(vocab, train_data, lambdas=[0.2, 0.7, 0.1])

Training 3-gram language model
Training 2-gram language model
Training 1-gram language model


In [ ]:
lambda_grid = [
   [0.5, 0.4, 0.1],
    [0.4, 0.5, 0.1],
    [0.3, 0.6, 0.1],
    [0.2, 0.7, 0.1],
    [0.1, 0.8, 0.1],
]
# Train and evaluate the model for each set of lambda values
best_lambda = None
best_perplexity = float('inf')
for lambdas in lambda_grid:
    ilm = InterpolateNGramLM(3)
    ilm.train(vocab, train_data, lambdas=lambdas)
    perplexity = ilm.perplexity(valid_data)
    
    print(f"Lambdas: {lambdas}, Perplexity: {perplexity}")
    
    if perplexity < best_perplexity:
        best_lambda = lambdas
        best_perplexity = perplexity
        
# Train a final model with the best lambda values
final_model = InterpolateNGramLM(3)
final_model.train(vocab, train_data, lambdas=best_lambda)
test_perplexity = final_model.perplexity(test_data)
print(f"Test perplexity: {test_perplexity}")

Training 3-gram language model
Training 2-gram language model
Training 1-gram language model
Lambdas: [0.5, 0.4, 0.1], Perplexity: 126.26886448793077
Training 3-gram language model
Training 2-gram language model
Training 1-gram language model
Lambdas: [0.4, 0.5, 0.1], Perplexity: 121.91287165028098
Training 3-gram language model
Training 2-gram language model
Training 1-gram language model
Lambdas: [0.3, 0.6, 0.1], Perplexity: 119.58096039943413
Training 3-gram language model
Training 2-gram language model
Training 1-gram language model
Lambdas: [0.2, 0.7, 0.1], Perplexity: 119.11642110884343
Training 3-gram language model
Training 2-gram language model
Training 1-gram language model
Lambdas: [0.1, 0.8, 0.1], Perplexity: 121.09118796605253
Training 3-gram language model
Training 2-gram language model
Training 1-gram language model
Test perplexity: 97.66795707054055


In [ ]:
print(ilm.perplexity(valid_data))
print(ilm.perplexity(test_data))

121.09118796605253
99.44721686710868
